<h1>Demo 1: QCS EVALUATION </h1>
<a id="pipeline_1"></a>


<p>Here you can evaluate your Quality Control Standards for no normalization data, Total Ion Ccount normalization data, and Internal Standard normalization data. For Pipeline #1, we will evaluate Quality Control Standard using Relative Standard Deviation (RSD) calculation and will have visualization of intensity plot, violin plot of Quality Control Standards to assess their variation throughout intra/interday batch. You can change the file path and file name accordingly and change the sample_set based on the names of your tissue samples in your dataset. It is recommended to run the pipeline in order then go to certain parts to look at the results again</p>

1. [Input](#input_1)
2. [No Normalization Dataset (input)](#no_norm_1)
3. [TIC Normalization Dataset (input)](#TIC_norm_1)
4. [IS Normalization Dataset (will be created)](#IS_norm_1)
5. [Visualization](#visualization_1) 

## Packages to Download ##

In [ ]:
## Packages for RSD overview table ##
install.packages("htmltools")
install.packages("kableExtra")
install.packages("IRdisplay")

## Packages for Violin plot ##
install.packages("ggplot2")
install.packages("dplyr")
install.packages("RColorBrewer")

## Package for excel sheet creating ##
install.packages("openxlsx")

cat("\033[1mSuccessfully Downloaded Packages\033[0m\n")

## Input Information ##
<a id="input_1"></a>
1) <b>Data_matrix_no_normalization </b> (csv file obtained from SciLs lab directly)
2) <b>Data_matrix_TIC_normalization </b> (csv file obtained from SciLs lab directly)
3) <b>QCS Analytes m/z values (ie: propranolol 260.16, D7-propranolol 267.14)</b> (input value *if using VS code, input at the top bar)
4) <b>Batch_info (csv file with batch information for sample construction)</b> (check the given example file *must have "injection.order" and "batch" column)
5) <b>Sample_set (tissue sample names, check the example given)</b>

In [ ]:
### Please edit the file path, file name ###
no_normalization_data_1 <- read.csv("input_1/119feature_T&QCS_BaseRemoval_noNorm.csv", # check file path
                                  header = FALSE,
                                  check.name = FALSE,
                                  stringsAsFactors = FALSE)

TIC_normalization_data_1 <- read.csv("input_1/119feature_T&QCS_BaseRemoval_TICNorm.csv", # check file path
                                   header = FALSE,
                                   check.name = FALSE,
                                   stringsAsFactors = FALSE) 

propranolol_mz_value_1 <- as.numeric(readline("Enter Propranolol m/z value: ")) ## ie: 260.186

d7_propranolol_mz_value_1 <- as.numeric(readline("Enter D7-propranolol m/z value: ")) ## ie: 267.187

batch_info_1 <- read.csv("input_1/batch_info_TIC.csv", # check file path
                       header = TRUE,
                       check.name = FALSE,
                       stringsAsFactors = FALSE) 

cat("\033[1mChange the sample_set accordingly to the dataset!\033[0m\n")
sample_set_1 <- c("ChickenHeart", "ChickenLiver", "GoatLiver") # please check your sample set if they are correctly named

In [ ]:
# Check if "injection.order" column exists
if ("injection.order" %in% colnames(batch_info_1)) {
    cat("\033[1mColumn 'injection.order' exists\033[0m\n")
} else {
    cat("\033[1mColumn 'injection.order' does not exist. Please check/edit colname for batch_info\033[0m\n")
}

if ("batch" %in% colnames(batch_info_1)) {
    cat("\033[1mColumn 'batch' exists.\033[0m\n")
} else {
    cat("\033[1mColumn 'batch' does not exist. Please check/edit colname for batch_info\033[0m\n")
}

<h2><u><b>No normalization dataset</b></u></h2>
<a id="no_norm_1"></a>

In [ ]:
### Making formatted table no_normalization ###
# Read the CSV file
rawdata <- no_normalization_data_1
mz_header <- unlist(strsplit(as.character(rawdata[9, 1]), ";"))[1]
peak_header <- unlist(strsplit(as.character(rawdata[9, 1]), ";"))[5:length(unlist(strsplit(as.character(rawdata[9, 1]), ";")))]
peak <- unlist(strsplit(as.character(rawdata[10, 1]), ";"))[5:length(unlist(strsplit(as.character(rawdata[9, 1]), ";")))]

mz_data_list <- list()
peak_data_list <- list()
combined_batch_data_list <- list()

# Extract data into datalist
for (i in 10:nrow(rawdata)){
  mz <- unlist(strsplit(as.character(rawdata[[i, 1]]), ";"))[1]
  peak <- unlist(strsplit(as.character(rawdata[i, 1]), ";"))[5:length(unlist(strsplit(as.character(rawdata[9, 1]), ";")))]
  mz_data_list[[i - 9]] <- mz
  peak_data_list[[i - 9]] <- peak
  combined_batch_data <- c(mz, peak)
  combined_batch_data_list[[i - 9]] <- combined_batch_data
}

# Combine data into a data frame
combined_batch_data_df <- as.data.frame(do.call(rbind, combined_batch_data_list), stringsAsFactors = FALSE)

# Set column names as mz 
colnames(combined_batch_data_df) <- c("mz", peak_header)

# make name column (will be same as mz column)
combined_batch_data_df$name <- combined_batch_data_df$mz 

# Set 'rt' column to a constant value
combined_batch_data_df$rt <- 666 # Assigning rt a random number

# Reorder columns again
combined_batch_data_df <- combined_batch_data_df[, c("name", "mz", "rt", peak_header)]

# check if each sample set exist in columns
sample_set_exist <- logical(length(sample_set_1))

# Check if each sample set exists in any of the column names
for (i in seq_along(sample_set_1)) {
  sample_set_exist[i] <- any(grepl(sample_set_1[i], colnames(combined_batch_data_df)))
}

# Identify sample sets that do not exist in any column names
missing_sample_sets <- sample_set_1[!sample_set_exist]

if (length(missing_sample_sets) > 0) {
  message("Error: The following sample sets do not exist in any column names: ", paste(missing_sample_sets, collapse = ", ", "check your sample_set"))
} else {
  message("Successfully made formatted table. All sample sets exist in at least one column name.")
}

# Check if m/z values exist in rows
propranolol_mz_exist <- any(combined_batch_data_df$mz == propranolol_mz_value_1)
d7_propranolol_mz_exist <- any(combined_batch_data_df$mz == d7_propranolol_mz_value_1)

# Print messages based on existence
if (!propranolol_mz_exist) {
  message("Error: The propranolol m/z value ", propranolol_mz_value_1, " does not exist in any row.")
} else {
  message("The propranolol m/z value ", propranolol_mz_value_1, " exists in at least one row.")
}

if (!d7_propranolol_mz_exist) {
  message("Error: The D7-propranolol m/z value ", d7_propranolol_mz_value_1, " does not exist in any row.")
} else {
  message("The D7-propranolol m/z value ", d7_propranolol_mz_value_1, " exists in at least one row.")
}

# uncomment to check if combined_batch_data_df looks correct and if sample names are correct
#print(combined_batch_data_df)
#print(colnames(combined_batch_data_df))

# making as a formatted csv file
write.csv(combined_batch_data_df,
          file = "dataset_1/formatted_no_norm_batch_data.csv",
          row.names = FALSE)

## Make batch info combined dataframe ##
# dataframe with injection order, batch info, batch data
combined_batch_data_df_transposed <- as.data.frame(t(combined_batch_data_df[, -(1:3)]))
colnames(combined_batch_data_df_transposed) <- combined_batch_data_df[, 1]
combined_batch_info_data <- cbind(batch_info_1[,-1], combined_batch_data_df_transposed) 

# making as a formatted csv file
write.csv(combined_batch_info_data,
          file = "dataset_1/formatted_no_norm_combined_batch_data.csv",
          row.names = TRUE)

cat("\033[1mSuccessfully formmated no normalized datasets (check dataset_1)\033[0m\n")

## RSD Calculation (No Normalization) ##

In [ ]:
# functions 
# extract tissue samples away function
extract_qcs_data <- function(dataset, sample_set) {
  pattern <- paste(sample_set, collapse = "|")
  
  # Extract tissue rows
  tissue_rows <- grep(pattern, rownames(dataset))
  
  # Subset dataframe to exclude tissue rows
  qcs_data <- dataset[-tissue_rows, , drop = FALSE]
  
  return(qcs_data)
}

In [ ]:
## QCS RSD Calculation ##
# extract tissue samples away function
extract_qcs_data <- function(dataset, sample_set) {
  pattern <- paste(sample_set, collapse = "|")
  
  # Extract tissue rows
  tissue_rows <- grep(pattern, rownames(dataset))
  
  # Subset dataframe to exclude tissue rows
  qcs_data <- dataset[-tissue_rows, , drop = FALSE]
  
  return(qcs_data)
}

# get rid of tissue samples
qcs_batch_info_data <- extract_qcs_data(combined_batch_info_data, sample_set_1)
batch_info_columns <- qcs_batch_info_data[, 1:2, drop = FALSE]

# extract qcs data
propranolol_peak_data <- qcs_batch_info_data[,colnames(qcs_batch_info_data) == propranolol_mz_value_1, drop = FALSE]
propranolol_data <- cbind(batch_info_columns, propranolol_peak_data)

d7_propranolol_peak_data <- qcs_batch_info_data[,colnames(qcs_batch_info_data) == d7_propranolol_mz_value_1, drop = FALSE]
d7_propranolol_data <- cbind(batch_info_columns, d7_propranolol_peak_data)

# Function to calculate RSD
calculate_rsd <- function(data) {
  sd_value <- sd(data)
  mean_value <- mean(data)
  rsd <- (sd_value / mean_value) * 100
  return(rsd)
}

# Function to calculate RSD for each batch
calculate_batch_rsd <- function(data, mz_value) {
  # Convert mz_value to character
  mz_value <- as.character(mz_value)
  # Convert column to numeric
  data[[mz_value]] <- as.numeric(data[[mz_value]], na.rm = TRUE)
  
  # Calculate RSD for each batch, rounding to two decimal places
  batch_rsd <- aggregate(data[[mz_value]],
                         by = list(batch = data$batch),
                         FUN = function(x) round(calculate_rsd(x), 2))
  
  # Give columns as batch and RSD
  colnames(batch_rsd) <- c("Batch", "RSD")
  
  # Calculate interday RSD, rounding to two decimal places
  interday_rsd <- round(calculate_rsd(data[[mz_value]]), 2)
  
  # Create a new row for interday RSD
  interday_row <- data.frame(Batch = "Interday", RSD = interday_rsd)
  
  # Combine batch_rsd with interday_rsd
  batch_rsd <- rbind(batch_rsd, interday_row)
  
  # Add percentage symbol
  batch_rsd$RSD <- paste0(batch_rsd$RSD, "%")
  
  # Replace batch labels with dynamic intraday labels
  batch_rsd$Batch <- ifelse(batch_rsd$Batch == "Interday", "Interday", 
                            paste0("Intraday ", as.character(batch_rsd$Batch)))
  
  return(batch_rsd)
}

# usage:
rsd_propranolol <- calculate_batch_rsd(propranolol_data, propranolol_mz_value_1)
cat("\033[1mSucessfully calculated rsd (propranolol) for no normalized dataset.\033[0m\n")
cat("m/z value:", propranolol_mz_value_1, "\n")
print(rsd_propranolol)
                           
rsd_d7_propranolol <- calculate_batch_rsd(d7_propranolol_data, d7_propranolol_mz_value_1)
cat("\033[1mSucessfully calculated rsd (d7_propranolol) for no normalized dataset.\033[0m\n")
cat("m/z value:", d7_propranolol_mz_value_1, "\n")
print(rsd_d7_propranolol)